In [74]:
import pandas as pd
import pyodbc
import sqlite3

# iedereen heeft andere servername btw

In [75]:
servername = 'LAPTOP-RPEMJO16\\SQLEXPRESS'

# Northwind connectie

In [76]:
northwind_conn = pyodbc.connect('DRIVER={SQL SERVER};SERVER='+servername+';DATABASE=NorthWind; Trusted_Connection=yes')
nwCursor = northwind_conn.cursor()
nwCursor

# Adventureworks connectie

In [77]:
adventureworks_conn = pyodbc.connect('DRIVER={SQL SERVER};SERVER='+servername+';DATABASE=AdventureWorks2019; Trusted_Connection=yes')
awCursor = adventureworks_conn.cursor()
awCursor

# Aenc connectie

In [78]:
aenc_conn = pyodbc.connect('DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};DBQ=../raw-databases/aenc.accdb;')
aencCursor = aenc_conn.cursor()

# Dataframes Aanmaken

### aenc

In [80]:
aE_Customer = pd.read_sql_query('SELECT * FROM customer', aenc_conn)
aE_SalesOrderItem = pd.read_sql_query('SELECT * FROM sales_order_item', aenc_conn)
aE_SalesOrder = pd.read_sql_query('SELECT * FROM sales_order', aenc_conn)

C:\Users\kwand\AppData\Local\Temp\ipykernel_12152\1986022038.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  aE_Customer = pd.read_sql_query('SELECT * FROM customer', aenc_conn)
C:\Users\kwand\AppData\Local\Temp\ipykernel_12152\1986022038.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  aE_SalesOrderItem = pd.read_sql_query('SELECT * FROM sales_order_item', aenc_conn)
C:\Users\kwand\AppData\Local\Temp\ipykernel_12152\1986022038.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  aE_SalesOrder = pd.read_sql_query('SEL

### AdventureWorks

In [81]:
Prod_Prod = pd.read_sql_query('SELECT * FROM Production.Product', adventureworks_conn)
Prod_Inv = pd.read_sql_query('SELECT * FROM Production.ProductInventory', adventureworks_conn)
Sales_Sales = pd.read_sql_query('SELECT * FROM Sales.SalesOrderDetail', adventureworks_conn)
Sales_OrderHeader = pd.read_sql_query('SELECT * FROM Sales.SalesOrderHeader', adventureworks_conn)

av1 = pd.merge(Prod_Prod, Prod_Inv, how='outer', on=['ProductID'])

C:\Users\kwand\AppData\Local\Temp\ipykernel_12152\3137805353.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  Prod_Prod = pd.read_sql_query('SELECT * FROM Production.Product', adventureworks_conn)
C:\Users\kwand\AppData\Local\Temp\ipykernel_12152\3137805353.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  Prod_Inv = pd.read_sql_query('SELECT * FROM Production.ProductInventory', adventureworks_conn)
C:\Users\kwand\AppData\Local\Temp\ipykernel_12152\3137805353.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  Sales_Sa

### Northwind

In [82]:
nw_Emp = pd.read_sql_query('SELECT * FROM Employees', northwind_conn)
nw_Ord = pd.read_sql_query('SELECT * FROM Orders', northwind_conn)
nw_OrderDetails = pd.read_sql_query('SELECT * FROM [Order Details]', northwind_conn)

EmpxOrd = pd.merge(nw_Emp, nw_Ord, how='outer', on='EmployeeID')

C:\Users\kwand\AppData\Local\Temp\ipykernel_12152\1119212295.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  nw_Emp = pd.read_sql_query('SELECT * FROM Employees', northwind_conn)
C:\Users\kwand\AppData\Local\Temp\ipykernel_12152\1119212295.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  nw_Ord = pd.read_sql_query('SELECT * FROM Orders', northwind_conn)
C:\Users\kwand\AppData\Local\Temp\ipykernel_12152\1119212295.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  nw_OrderDetails = pd.read_sql_query('SELECT * FROM [O

In [85]:
orderdetails1 = pd.merge(nw_OrderDetails, av1 , how='outer', on=['ProductID', 'Quantity'])
orderdetails2 = pd.merge(orderdetails1, EmpxOrd, how='outer', on='OrderID')
aE_customer = aE_Customer.rename(columns={'id': 'CustomerID'}, inplace=True)
orderdetails3 = pd.concat([aE_Customer, orderdetails2], axis=0)
od = orderdetails3[['OrderID', 'Quantity', 'ListPrice', 'OrderDate', 'ShippedDate', 'ProductID', 'EmployeeID', 'CustomerID']]
od = od[od['OrderID'].notna()]
od.columns

Index(['OrderID', 'Quantity', 'ListPrice', 'OrderDate', 'ShippedDate',
       'ProductID', 'EmployeeID', 'CustomerID'],
      dtype='object')

### Export

In [90]:
od.dtypes
od['id'] = range(1, len(od) + 1)

,ListPrice


In [91]:
# Convert types
od['id'] = od['id'].fillna(0).astype(int)
od['OrderID'] = od['OrderID'].fillna(0).astype(int)
od['Quantity'] = od['Quantity'].fillna(0).astype(int)
od['ListPrice'] = od['ListPrice'].fillna(0).astype(float)
od['OrderDate'] = pd.to_datetime(od['OrderDate']) 
od['ShippedDate'] = pd.to_datetime(od['ShippedDate']) 
od['ProductID'] = od['ProductID'].fillna(0).astype(int)
od['EmployeeID'] = od['EmployeeID'].fillna(0).astype(int)
od['CustomerID'] = od['CustomerID'].fillna(0).astype(str)
od.dtypes

OrderID                 int32
Quantity                int32
ListPrice             float64
OrderDate      datetime64[ns]
ShippedDate    datetime64[ns]
ProductID               int32
EmployeeID              int32
CustomerID             object
id                      int32
dtype: object

In [92]:
od

,OrderID,Quantity,ListPrice,OrderDate,ShippedDate,ProductID,EmployeeID,CustomerID,id
0,10248,12,0.0,1996-07-04,1996-07-16,11,5,VINET,1
1,10248,10,0.0,1996-07-04,1996-07-16,42,5,VINET,2
2,10248,5,0.0,1996-07-04,1996-07-16,72,5,VINET,3
3,10249,9,0.0,1996-07-05,1996-07-10,14,6,TOMSP,4
4,10249,40,0.0,1996-07-05,1996-07-10,51,6,TOMSP,5
...,...,...,...,...,...,...,...,...,...
2150,11077,2,0.0,1998-05-06,NaT,64,1,RATTC,2151
2151,11077,1,0.0,1998-05-06,NaT,66,1,RATTC,2152
2152,11077,2,0.0,1998-05-06,NaT,73,1,RATTC,2153
2153,11077,4,0.0,1998-05-06,NaT,75,1,RATTC,2154


In [70]:
database = "UnitedOutdoors"
export_conn = pyodbc.connect("DRIVER={SQL SERVER};SERVER="+servername + ";DATABASE="+database+";Trusted_Connection=yes")
export_cursor = export_conn.cursor()
export_cursor

ProgrammingError: ('42000', '[42000] [Microsoft][ODBC SQL Server Driver][SQL Server]Cannot open database "UnitedOutdoors" requested by the login. The login failed. (4060) (SQLDriverConnect); [42000] [Microsoft][ODBC SQL Server Driver][SQL Server]Cannot open database "UnitedOutdoors" requested by the login. The login failed. (4060)')

In [ ]:
for index, row in od.iterrows():
    try:
        query = """INSERT INTO order_details (
            order_detailsid
            order_id,
            product_quantity, 
            product_listprice,
            order_date,
            shipping_date,
            product_productid
            employee_employeeid,
            customer_id
        ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)"""  
        
        values = (
            row['id'],
            row['OrderID'],
            row['Quantity'],
            row['ListPrice'],
            row['OrderDate'],
            row['ShippedDate'],
            row['ProductID'],
            row['EmployeeID'],
            row['CustomerID'],

        )

        export_cursor.execute(query, values)

    except pyodbc.IntegrityError as e:
        continue
    except pyodbc.Error as e:
        print("Er is een fout opgetreden: ", e)
        print("Foutieve query: ", query)
    
export_conn.commit()